In [11]:
!pip install -U pip
!pip install bitsandbytes==0.39.0
!pip install torch==2.0.1
!pip install transformers==4.30.0
!pip install accelerate==0.20.3
!pip install git+https://github.com/huggingface/peft.git
!pip install datasets==2.12.0
!pip install loralib==0.1.1
!pip install einops==0.6.1
!pip install langchain
!pip install xformers
!pip install gradio

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-6q637wn4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-6q637wn4
  Resolved https://github.com/huggingface/peft.git to commit 06fd06a4d2e8ed8c3a253c67d9c3cb23e0f497ad
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 76.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.4/288.4 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import langchain
from langchain.llms import HuggingFacePipeline
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain
import torch
import transformers
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

In [3]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
DEVICE = 'cuda:0'
COLAB_PATH = None

### If on google colab, uncomment the following ###
from google.colab import drive
import locale
locale.getpreferredencoding = lambda: "UTF-8"
drive.mount('/content/drive')
COLAB_PATH = '/content/drive/MyDrive/gntLM/FinalGNTLM'
###

# default inference on proposed model
EXPERIMENT = 'experiment_4'

Mounted at /content/drive


In [4]:
def get_local_peft_model(experiment_name: str=EXPERIMENT, initial: bool=True) -> str:
    map_experiment_name_to_model = {
          'experiment_1': 'experiment_1_model',
          'experiment_2': 'experiment_2_model',
          'experiment_3': 'experiment_3_model',
          'experiment_4': 'experiment_4_model'
    }
    try:
      return map_experiment_name_to_model[experiment_name]
    except KeyError:
      if initial:
        raise KeyError(f'Please choose a name in {list(map_experiment_name_to_model.keys())}. If you want to load a different model, please provide the model directory and set initial=False')
      else:
        return experiment_name


def normalize_response(chat_chain: langchain.chains.ConversationChain, query: str) -> str:
    chat_chain.predict(input=query)

    chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.split('\n\n')[0]
    chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.strip()

    for stop_text in ['Human:', 'AI:']:
        chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.removesuffix(stop_text)

    chat.memory.chat_memory.messages[-1].content = chat.memory.chat_memory.messages[-1].content.strip()

    return chat_chain.memory.chat_memory.messages[-1].content

In [5]:
GNTLM = get_local_peft_model()
if COLAB_PATH:
  PEFT_MODEL = f'{COLAB_PATH}/{GNTLM}'
else:
  PEFT_MODEL = f'/{GNTLM}'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16
)

config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True,
    load_in_4bit=True
)

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.bos_token_id = 1

model = PeftModel.from_pretrained(model, PEFT_MODEL)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
stop_token_ids = [
    tokenizer.convert_tokens_to_ids(x) for x in [
        ['Human', ':'], ['AI', ':']
    ]
]

stop_token_ids = [torch.LongTensor(x).to(DEVICE) for x in stop_token_ids]

In [7]:
class StopOnTokens(transformers.StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
                return True
        return False

In [8]:
stopping_criteria = transformers.StoppingCriteriaList([StopOnTokens()])

In [16]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,
    task='text-generation',
    stopping_criteria=stopping_criteria,
    temperature=0.7,
    top_p=0.7,
    top_k=0,
    max_new_tokens=128,
    repetition_penalty=1.5
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RobertaPreLayerN

In [10]:
prompt = langchain.PromptTemplate(
    input_variables=["instruction"],
    template="{instruction}"
)

llm = HuggingFacePipeline(pipeline=generate_text)
llm_chain = langchain.LLMChain(llm=llm, prompt=prompt)

memory = ConversationBufferWindowMemory(
    memory_key="history",
    k=5,
    return_only_outputs=True
)

chat = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=False
)

chat.prompt.template = \
"""The following is a conversation between a human and an guidance counseling AI. The AI is talkative and provides lots of specific details from its context and focuses on answering the question posed by the human. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:"""

In [ ]:
normalize_response(chat, 'I have a strong sense of justice and can persuade people with my speech prowess. What should my future occupation be?')

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1259: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'You are very persuasive and passionate about your beliefs. Your ability to communicate effectively makes you well suited for public speaking or debate competitions. Perhaps becoming a lawyer or politician would also suit you as these careers often require persuasion skills. However, if you prefer working independently rather than in teams, consider being a writer or consultant. These occupations typically involve communicating ideas and persuading others through written text or spoken words respectively.'

In [ ]:
normalize_response(chat, 'Could you please give me a couple more choices? I do not really want to become a politician.')

Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


'Based on your current interests and passions, there are several other career options that may suit you better than being either a lawyer or politician. Some alternatives include working as a journalist or author, both roles requiring excellent writing abilities and knowledge about societal issues. Another option would be joining a non-profit organization where you could use your persuasion skills to advocate for specific causes. Finally, if you enjoy public speaking and debating topics related to law and politics, perhaps teaching or consulting might interest you.'